import yfinance as yf
msft = yf.Ticker('MSFT')
msft.info


In [ ]:
import yfinance as yf

msft = yf.Ticker("MSFT")
msft.info

In [ ]:
a= msft.history(period='1mo')

In [ ]:
a

In [ ]:
import yfinance as yf 
hdfc =yf.Ticker('HDFCBANK.NS')
hdfc.history(period='1mo',interval='15m')


In [ ]:
import pandas as pd

def calculate_ema_vwap(df, ema_window, vwap_period):
    df['EMA'] = df['Close'].ewm(span=ema_window, adjust=False).mean()
    df['Cumulative_Volume'] = df['Volume'].cumsum()
    df['Cumulative_Value'] = (((df['Close']+df['Low']+df['High'])/3) * df['Volume']).cumsum()
    df['VWAP'] = df['Cumulative_Value'] / df['Cumulative_Volume']
    df['Captured_Price'] = df.apply(lambda row: row['Close'] if row['EMA'] <= row['VWAP'] else None, axis=1)
    
    df['Index'] = range(1, len(df) + 1)

    df['Price'] = df['Close']

    return df

hdfc_data = pd.read_csv("HDFCBANK.csv")

ema_window_size = 15
vwap_period_size = 15

hdfc_data_ema_vwap = calculate_ema_vwap(hdfc_data, ema_window_size, vwap_period_size)

print(hdfc_data_ema_vwap)

hdfc_data_ema_vwap.to_csv("hdfc.csv", index=False)


In [ ]:
import pandas as pd

def calculate_ema_vwap(df, ema_window, vwap_period, chunk_size):
    results = []
    for i in range(0, len(df), chunk_size):
        chunk = df.iloc[i:i+chunk_size].copy()

        chunk['EMA'] = chunk['Close'].ewm(span=ema_window, adjust=False).mean()
        chunk['Cumulative_Volume'] = chunk['Volume'].cumsum()
        chunk['Cumulative_Value'] = (((chunk['Close']+chunk['Low']+chunk['High'])/3) * chunk['Volume']).cumsum()
        chunk['VWAP'] = chunk['Cumulative_Value'] / chunk['Cumulative_Volume']
        chunk['Captured_Price'] = chunk.apply(lambda row: row['Close'] if row['EMA'] <= row['VWAP'] else None, axis=1)

        chunk['Index'] = range(1, len(chunk) + 1)
        chunk['Price'] = chunk['Close']

        results.append(chunk)

    return pd.concat(results)

# Read the data in chunks
chunk_size = 25
hdfc_data_chunks = pd.read_csv("HDFCBANK.csv", chunksize=chunk_size)

ema_window_size = 15
vwap_period_size = 15

# Process each chunk and concatenate the results
hdfc_data_ema_vwap = pd.concat([calculate_ema_vwap(chunk, ema_window_size, vwap_period_size, chunk_size) for chunk in hdfc_data_chunks])

print(hdfc_data_ema_vwap)

hdfc_data_ema_vwap.to_csv("hdfc.csv", index=False)
